# VacationPy


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities=pd.read_csv('../WeatherPy/output_data/cities.csv')
cities.head()

,City,Latitude,Longitude,Max-Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.1203,-134.9692,78.01,72,100,6.04,PF,1619495166
1,mataura,-46.1927,168.8643,46.00,75,91,5.01,NZ,1619495167
2,quelimane,-17.8786,36.8883,75.20,94,20,2.30,MZ,1619495167
3,hamilton,39.1834,-84.5333,63.00,48,75,9.22,US,1619495060
4,atuona,-9.8000,-139.0333,79.70,74,94,17.45,PF,1619495167


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = cities[['Latitude', 'Longitude']].astype(float)
weight = cities['Humidity'].astype(float)
center = (35,0)

fig = gmaps.figure(center=center, zoom_level=1.4)

heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, 
                                 max_intensity=80, point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
desirable_cities = cities.loc[(cities['Max-Temp'] > 70) &
                             (cities['Max-Temp'] < 90) &
                             (cities['Humidity'] < 30) &
                             (cities['Wind Speed'] < 10)].copy()
desirable_cities.reset_index(drop=True, inplace=True)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame()
hotel_df['Hotel Name'] = ''
hotel_df

,Hotel Name


In [13]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
params = {'radius': '5000',
          'type': 'lodging',
          'key': gkey}

for index,row in desirable_cities.iterrows():
    params['location'] = f'{row.Latitude},{row.Longitude}'
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index,'Hotel Name'] = response['results'][0]['name']
        hotel_df.loc[index, 'City'] = row.City
        hotel_df.loc[index, 'Country'] = row.Country
        hotel_df.loc[index, 'Latitude'] = row.Latitude
        hotel_df.loc[index, 'Longitude'] = row.Longitude
    except:
        print(f'No hotel found in {row.City}.')
        pass
    
hotel_df

No hotel found in bitkine.
No hotel found in taoudenni.
No hotel found in agadez.
No hotel found in kidal.
No hotel found in adrar.
No hotel found in tessalit.
No hotel found in goure.


,Hotel Name,City,Country,Latitude,Longitude
0,Expeditions Ténére voyages,bilma,NE,18.6853,12.9164
1,Bitkine,bitkine,TD,11.9801,18.2138
2,Oumou Elghoura,atar,MR,20.5169,-13.0499
3,Bungalows Guesthouse,tahoua,NE,14.8888,5.2692
4,"InterContinental Riyadh, an IHG Hotel",riyadh,SA,24.6877,46.7219
5,Bal uma sankar hotel and restaurant,ginda,IN,30.6945,78.4932
6,Hotel Montesclaros,choix,MX,26.7092,-108.3219
7,Taoudenni,taoudenni,ML,22.6783,-3.9836
9,Broadwater Mariner Resort,geraldton,AU,-28.7667,114.6000
10,Khumson Sanatoriy,gazalkent,UZ,41.5581,69.7708


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [16]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))